<a href="https://colab.research.google.com/github/ulubeykhuja/Amaliyot/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.9.0 which is incompatible.
cudf-cu12 

In [2]:
! pip install -U transformers

In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")
small_train = dataset["train"].shuffle(seed=42).select([i for i in list(range(1000))])
small_test = dataset["test"].shuffle(seed=42).select([i for i in list(range(300))])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=256)

tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_test = small_test.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
    )
trainer.train()

Step,Training Loss
10,0.698800
20,0.613400
30,0.523300
40,0.425800
50,0.309200
60,0.374700
70,0.503100
80,0.292000
90,0.223300
100,0.276400


TrainOutput(global_step=189, training_loss=0.2907527805793853, metrics={'train_runtime': 75.4888, 'train_samples_per_second': 39.741, 'train_steps_per_second': 2.504, 'total_flos': 198701097984000.0, 'train_loss': 0.2907527805793853, 'epoch': 3.0})

In [9]:
trainer.evaluate()

{'eval_loss': 0.42698293924331665,
 'eval_runtime': 2.1355,
 'eval_samples_per_second': 140.482,
 'eval_steps_per_second': 8.897,
 'epoch': 3.0}

In [19]:
texts = [
    'This movie was absolutely lovely',
    'Theis movie not bad',
    'Just ok movie',
    'film is very bad'
    ]


In [12]:
import torch
import torch.nn.functional as F

In [20]:
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=256).to('cuda')

with torch.no_grad():
  outputs = model(**inputs)
  probs = F.softmax(outputs.logits, dim=1)
  predictions = torch.argmax(probs, dim=1)

label_map = {0: 'NEGATIVE', 1: 'POSITIVE'}

for text, pred, prob in zip(texts, predictions, probs):
  print(text, label_map[pred.item()], prob[pred.item()])

This movie was absolutely lovely POSITIVE tensor(0.9865, device='cuda:0')
Theis movie not bad NEGATIVE tensor(0.8451, device='cuda:0')
Just ok movie POSITIVE tensor(0.7556, device='cuda:0')
film is very bad NEGATIVE tensor(0.9853, device='cuda:0')
